In [15]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting map library

print('Folium installed')
print('All Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
All Libraries imported.


In [16]:
CLIENT_ID='NSWC0SRL1NVQY551AP3I0RAQPIFKF3PXC3XBYGT1DEGC0CJM'
CLIENT_SECRET='1EDCF3ZSYUHAO2A03AOK0QEIIWM1UGSL3XBSYKTV4LYHAWV2'
latitude=43.6427
longitude=-79.3888
VERSION=20200401
search_query = 'restaurants'
radius = 10000 # 10 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_restaurants = requests.get(url).json()
result_restaurants

{'meta': {'code': 200, 'requestId': '5e9716266d8c56001bf2e07b'},
 'response': {'venues': [{'id': '53a07ba3498ee8946e98a7de',
    'name': 'Marigold Indian Bistro | Indian Restaurants in Toronto',
    'location': {'address': '552 Mt Pleasant',
     'lat': 43.64430171166487,
     'lng': -79.39000236526671,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64430171166487,
       'lng': -79.39000236526671}],
     'distance': 202,
     'postalCode': 'M4S 2M6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['552 Mt Pleasant', 'Toronto ON M4S 2M6', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16e941735',
      'name': 'Fast Food Restaurant',
      'pluralName': 'Fast Food Restaurants',
      'shortName': 'Fast Food',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586959980',
    'hasPerk': False},
   {

In [17]:
# assign relevant part of JSON to hotel
restaurants = result_restaurants['response']['venues']

# tranform hotels into a dataframe
df_restaurants = json_normalize(restaurants)
df_restaurants.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,53a07ba3498ee8946e98a7de,Marigold Indian Bistro | Indian Restaurants in...,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1586959980,False,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",202,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,NaN
1,4d14c68e380a8eec7b2185fa,Imago Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586959980,False,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",602,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,NaN
2,4b295e10f964a520ba9d24e3,Richtree Natural Market Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586959980,False,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",1301,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,NaN
3,55eba3b8498e808c93025230,Restaurants Canada,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1586959980,False,1155 Queen Street West,43.642628,-79.425337,"[{'label': 'display', 'lat': 43.64262834138462...",2943,M6J 1J4,CA,Toronto,ON,Canada,"[1155 Queen Street West, Toronto ON M6J 1J4, C...",NaN,NaN
4,4d8a7d78bc848cfabd63db2b,Valens Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586959980,False,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",1526,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,NaN


In [18]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_restaurants.columns if col.startswith('location.')] + ['id']
df_restaurants_filtered = df_restaurants.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_restaurants_filtered['categories'] = df_restaurants_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_restaurants_filtered.columns = [column.split('.')[-1] for column in df_restaurants_filtered.columns]

pd.DataFrame(df_restaurants_filtered)
df_restaurants_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",202,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,53a07ba3498ee8946e98a7de
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",602,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,4d14c68e380a8eec7b2185fa
2,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",1301,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,4b295e10f964a520ba9d24e3
3,Restaurants Canada,Office,1155 Queen Street West,43.642628,-79.425337,"[{'label': 'display', 'lat': 43.64262834138462...",2943,M6J 1J4,CA,Toronto,ON,Canada,"[1155 Queen Street West, Toronto ON M6J 1J4, C...",NaN,55eba3b8498e808c93025230
4,Valens Restaurants,Restaurant,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",1526,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,4d8a7d78bc848cfabd63db2b
5,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,"[{'label': 'display', 'lat': 43.64595052717185...",370,M5V 2C7,CA,Toronto,ON,Canada,"[200 Wellington St West, Unit 340, Toronto ON ...",NaN,4b5f325ff964a52093ac29e3
6,Subway,Sandwich Place,"88 Queens Quay West, Unit Fc-8",43.641058,-79.378586,"[{'label': 'display', 'lat': 43.64105847366237...",842,M5J 0B8,CA,Toronto,ON,Canada,"[88 Queens Quay West, Unit Fc-8, Toronto ON M5...",NaN,59f7478a6bd36b5ffec10c9d
7,Subway,Sandwich Place,287 King St. W,43.646459,-79.389506,"[{'label': 'display', 'lat': 43.64645929083939...",422,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4c7e90a5d51ea1430508589f
8,Oliver & Bonacini Restaurants,Office,2323 Yonge St.,43.708278,-79.398480,"[{'label': 'display', 'lat': 43.70827849987401...",7341,NaN,CA,Toronto,ON,Canada,"[2323 Yonge St. (Roehampton Ave.), Toronto ON,...",Roehampton Ave.,50be0f7ee4b0624f8d0b12ea
9,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,"[{'label': 'display', 'lat': 43.65119366354807...",1014,M5H 3M9,CA,Toronto,ON,Canada,"[123 Queen Street West, Suite # C72 (At Sherat...","At Sheraton Toronto Centre, Concourse",50aff967e4b07ba2ce4a1c3c


In [19]:
df_restaurants_filtered = df_restaurants_filtered[df_restaurants_filtered.categories.isin(['Fast Food','Restaurant','Korean Restaurant'])]
df_restaurants_filtered.categories = ('Food')
df_restaurants_filtered.reset_index(drop=True, inplace=True)
df_restaurants_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Imago Restaurants,Food,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",602,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,4d14c68e380a8eec7b2185fa
1,Richtree Natural Market Restaurants,Food,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",1301,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,4b295e10f964a520ba9d24e3
2,Valens Restaurants,Food,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",1526,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,4d8a7d78bc848cfabd63db2b
3,Myoung Dong Soon To Fu Korean Restaurant (Sush...,Food,620 Bloor St. W,43.664562,-79.413909,"[{'label': 'display', 'lat': 43.664562, 'lng':...",3164,NaN,CA,Toronto,ON,Canada,"[620 Bloor St. W, Toronto ON, Canada]",NaN,4bf342a16a31d13a1c83942e


In [20]:
df_restaurants_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
df_restaurants_filtered

,name,categories,address,lat,lng,distance,cc,formattedAddress,id
0,Imago Restaurants,Food,"14 Duncan Street, Suite 203",43.647910,-79.390810,602,CA,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",4d14c68e380a8eec7b2185fa
1,Richtree Natural Market Restaurants,Food,14 Queen St W,43.652614,-79.380231,1301,CA,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",4b295e10f964a520ba9d24e3
2,Valens Restaurants,Food,19 Baldwin Street,43.656096,-79.392839,1526,CA,"[19 Baldwin Street, Toronto ON, Canada]",4d8a7d78bc848cfabd63db2b
3,Myoung Dong Soon To Fu Korean Restaurant (Sush...,Food,620 Bloor St. W,43.664562,-79.413909,3164,CA,"[620 Bloor St. W, Toronto ON, Canada]",4bf342a16a31d13a1c83942e


In [23]:
allrestaurants_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the New York City

coordinates = [
    [43.772692,-79.504171],
    [43.740158,-79.216718],
    [43.638427,-79.382013],
    [43.648696,-79.562161],
    [43.772692,-79.504171]
]

my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
allrestaurants_map.add_child(my_PolyLine)

# add the Hotels as blue circle markers
for lat, lng, label in zip(df_restaurants_filtered.lat, df_restaurants_filtered.lng, df_restaurants_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(allrestaurants_map)

# display map
allrestaurants_map